If on google collab, if not skip the following cell

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/TFG_Matematicas'

Mounted at /content/gdrive
/content/gdrive/MyDrive/TFG_Matematicas


## Preparation to test models

In [2]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dense, Dropout, Input, SimpleRNN, LSTM, Input, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import plot_model
from utils.utils import *


In [3]:
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
# except ÇValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [4]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
train_idxs = read_indexes('./utils/train.txt')
validation_idxs =  read_indexes('./utils/validation.txt')
test_idxs = read_indexes('./utils/test.txt')

In [8]:
data_indexes = [(i+1,j+1) for i in range(10) for j in range(2000)]
parameters = load_parameter_list()
df_parameters = pd.DataFrame(parameters, columns=['cx', 'cy', 'a', 'b', 'theta', 'e1', 'e2'])
df_parameters['data_index'] = data_indexes

y_train = df_parameters[df_parameters['data_index'].isin(train_idxs)].drop('data_index', axis = 1)
y_validation = df_parameters[df_parameters['data_index'].isin(validation_idxs)].drop('data_index', axis = 1)
y_test = df_parameters[df_parameters['data_index'].isin(test_idxs)].drop('data_index', axis = 1)

In [9]:

X_train = np.array(load_data_multithreaded(train_idxs,100, small = True))
print('train loaded')
X_validation = np.array(load_data_multithreaded(validation_idxs,100, small = True))
print('validation loaded')
#X_test = np.array(load_data_multithreaded(test_idxs,100, small = True))
#print('test loaded')

train loaded
validation loaded


In [10]:
mean = np.mean(X_train.flatten())
std = np.std(X_train.flatten())

In [11]:
mean, std

(0.0028948491014701818, 0.015848718933305188)

# Preparing metrics

In [12]:
def e1_mse(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -2], output)
    return output_mae
def e2_mse(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -1], output)
    return output_mae

def e1_mae(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -2], output)
    return output_mae
def e2_mae(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -1], output)
    return output_mae

def e1_rmse(y_true, y_pred):
    output = y_pred[:, -2]
    output_rmse = rmse(y_true[:, -2], output)
    return output_rmse
def e2_rmse(y_true, y_pred):
    output = y_pred[:, -1]
    output_rmse = rmse(y_true[:, -1], output)
    return output_rmse

In [13]:
METRICS=[tf.keras.metrics.RootMeanSquaredError(name='rmse'),
         tf.keras.metrics.MeanAbsoluteError(name='mae'),
         e1_mse,e2_mse,e1_mae,e2_mae]


#### Libraries for hyperparameter tunning

In [14]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [15]:
import keras_tuner as kt

# Feedforward model

In [16]:
def model_builder(hp):

    # hyperparameter space
    layer_combinations = {
        2: [1024, 256],
        3: [1024, 256, 64],
        4: [1024, 256, 128, 64],
        5: [1024, 512, 256, 128, 64],
    }
    n = hp.Choice('layers', values=[2,3,4,5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    #lr = hp.Choice('learning_rate', values=[0.001,0.0001])
    activation = hp.Choice('activation', values=["relu","leaky_relu","elu"])

    # model construction
    feedforward = keras.Sequential()
    feedforward.add(Input(shape=(440, 17)))
    feedforward.add(Flatten())
    for i in layer_combinations[n]:
      feedforward.add(Dense(i, activation=activation))
      if dropout_boolean:
        feedforward.add(Dropout(0.2))
    feedforward.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    feedforward.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return feedforward

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    factor=3,
    hyperband_iterations=1,
    directory='hyperparameter_tunning',
    project_name='feedforward'
)

tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

Reloading Tuner from hyperparameter_tunning/feedforward/tuner0.json


In [17]:
tuner.results_summary()

Results summary
Results in hyperparameter_tunning/feedforward
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
layers: 5
dropout: 0
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.5097925662994385

Trial 0026 summary
Hyperparameters:
layers: 4
dropout: 0
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.5212552547454834

Trial 0025 summary
Hyperparameters:
layers: 5
dropout: 0
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0018
Score: 0.5324428677558899

Trial 0017 summary
Hyperparameters:
layers: 3
dropout: 0
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.536333441734314

Trial 0024 summary
Hyperparameters:
layers: 4
dropout: 0
activation: elu
tuner/epochs: 25
tuner/initial_

# Convolutional model

In [18]:
def model_builder(hp):

    # hyperparameter space
    conv_n = hp.Choice('conv_layers', values=[2,3])
    kernel_s =  hp.Choice('kernel_size', values=[3,5])
    kernel_sizes = {
        3: (3,3),
        5: (5,5)
    }
    layer_combinations = {
          5: [1024, 512, 256, 128, 64],
    }
    n = hp.Choice('layers', values=[5])
    pooling_layers = hp.Choice('pooling_layers', values=['max', 'avg'])
    transition_layer = hp.Choice('transition_layer', values=['g_max', 'g_avg', 'flatten'])
    activation = hp.Choice('activation', values=["relu"])
    dropout_boolean = hp.Choice('dropout', values=[True, False])

    # model construction
    cnn = keras.Sequential()
    cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(440, 17, 1), padding='same'))
    cnn.add(BatchNormalization()),
    if pooling_layers == 'max':
        cnn.add(MaxPooling2D((1, 2))),
    else:
      cnn.add(AveragePooling2D((1, 2))),

    for i in range(conv_n-1):
      cnn.add(Conv2D(32, kernel_sizes[kernel_s], activation='relu', padding='same'))
      cnn.add(BatchNormalization()),
      if pooling_layers == 'max':
        cnn.add(MaxPooling2D((1, 2))),
      else:
        cnn.add(AveragePooling2D((1, 2))),

    if transition_layer == 'g_max':
      cnn.add(GlobalMaxPooling2D()),
    elif transition_layer == 'g_avg':
      cnn.add(GlobalAveragePooling2D()),
    else:
      cnn.add(Flatten()),

    for i in layer_combinations[n]:
      cnn.add(Dense(i, activation=activation))
      if dropout_boolean:
        cnn.add(Dropout(0.2))
    cnn.add(Dense(7))

    #model compilation
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    cnn.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return cnn

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    factor=3,
    hyperband_iterations=2,
    directory='hyperparameter_tunning',
    project_name='cnn'
)

tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

Reloading Tuner from hyperparameter_tunning/cnn/tuner0.json


In [19]:
tuner.results_summary()

Results summary
Results in hyperparameter_tunning/cnn
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0025 summary
Hyperparameters:
conv_layers: 3
kernel_size: 3
layers: 5
pooling_layers: max
transition_layer: g_max
activation: relu
dropout: 0
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 0.4195246994495392

Trial 0024 summary
Hyperparameters:
conv_layers: 2
kernel_size: 5
layers: 5
pooling_layers: max
transition_layer: flatten
activation: relu
dropout: 0
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0019
Score: 0.45265188813209534

Trial 0047 summary
Hyperparameters:
conv_layers: 2
kernel_size: 3
layers: 5
pooling_layers: avg
transition_layer: flatten
activation: relu
dropout: 0
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 0.46596837043762207

Trial 0027 summary
Hyperparameters:
conv_layers: 3
kernel_size: 3
la

# Recurrent Neural Network with simple RNN layer

In [20]:
def model_builder(hp):

    # hyperparameter space
    rnn_layer_combinations = {
          1: [25],
          2: [20, 30],
      }
    rnn_n = hp.Choice('rnn_layers', values=[1,2])
    layer_combinations = {
        5: [1024, 512, 256, 128, 64],
    }

    n = hp.Choice('layers', values=[5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    sequences_boolean = hp.Choice('sequences', values=[True, False])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])

    # model construction
    if len(rnn_layer_combinations[rnn_n]) == 1:
      rnn = keras.Sequential()
      rnn.add(SimpleRNN(rnn_layer_combinations[rnn_n][0], input_shape=(440, 17), return_sequences=sequences_boolean))
      rnn.add(Flatten())
    else:
      rnn = keras.Sequential()
      rnn.add(SimpleRNN(rnn_layer_combinations[rnn_n][0], input_shape=(440, 17), return_sequences=True))
      rnn.add(SimpleRNN(rnn_layer_combinations[rnn_n][1], return_sequences=sequences_boolean))

    rnn.add(Flatten())
    for i in layer_combinations[n]:
      rnn.add(Dense(i, activation=activation))
      if dropout_boolean:
        rnn.add(Dropout(0.2))
    rnn.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    rnn.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return rnn

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    factor=3,
    hyperband_iterations=1,
    directory='hyperparameter_tunning',
    project_name='rnn'
)

tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

Trial 20 Complete [00h 24m 57s]
val_loss: 1.0972458124160767

Best val_loss So Far: 0.5282773971557617
Total elapsed time: 17h 06m 43s


In [21]:
tuner.results_summary()

Results summary
Results in hyperparameter_tunning/rnn
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
rnn_layers: 1
layers: 5
dropout: 0
sequences: 1
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.5282773971557617

Trial 0017 summary
Hyperparameters:
rnn_layers: 2
layers: 5
dropout: 0
sequences: 1
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.5547113418579102

Trial 0012 summary
Hyperparameters:
rnn_layers: 1
layers: 5
dropout: 0
sequences: 1
activation: relu
tuner/epochs: 9
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0001
Score: 0.6542760133743286

Trial 0015 summary
Hyperparameters:
rnn_layers: 2
layers: 5
dropout: 0
sequences: 1
activation: leaky_relu
tuner/epochs: 9
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0003
Score: 0.66

#  Recurrent Neural Network Model with LSTM

In [22]:
def model_builder(hp):

    # hyperparameter space
    lstm_layer_combinations = {
          1: [25],
          2: [20, 30],
      }
    lstm_n = hp.Choice('lstm_layers', values=[1,2])
    layer_combinations = {
        5: [1024, 512, 256, 128, 64],
    }

    n = hp.Choice('layers', values=[5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    sequences_boolean = hp.Choice('sequences', values=[True, False])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])

    # model construction
    if len(lstm_layer_combinations[lstm_n]) == 1:
      rnn_lstm = keras.Sequential()
      rnn_lstm.add(LSTM(lstm_layer_combinations[lstm_n][0], input_shape=(440, 17), return_sequences=sequences_boolean))
      rnn_lstm.add(Flatten())
    else:
      rnn_lstm = keras.Sequential()
      rnn_lstm.add(LSTM(lstm_layer_combinations[lstm_n][0], input_shape=(440, 17), return_sequences=True))
      rnn_lstm.add(LSTM(lstm_layer_combinations[lstm_n][1], return_sequences=sequences_boolean))

    rnn_lstm.add(Flatten())
    for i in layer_combinations[n]:
      rnn_lstm.add(Dense(i, activation=activation))
      if dropout_boolean:
        rnn_lstm.add(Dropout(0.2))
    rnn_lstm.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    rnn_lstm.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return rnn_lstm

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    factor=3,
    hyperband_iterations=1,
    directory='hyperparameter_tunning',
    project_name='rnn_lstm'
)



tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

Reloading Tuner from hyperparameter_tunning/rnn_lstm/tuner0.json


In [23]:
tuner.results_summary()

Results summary
Results in hyperparameter_tunning/rnn_lstm
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
lstm_layers: 1
layers: 5
dropout: 0
sequences: 1
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.5112601518630981

Trial 0017 summary
Hyperparameters:
lstm_layers: 2
layers: 5
dropout: 0
sequences: 1
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.5227682590484619

Trial 0015 summary
Hyperparameters:
lstm_layers: 1
layers: 5
dropout: 0
sequences: 1
activation: relu
tuner/epochs: 9
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0004
Score: 0.5995014905929565

Trial 0013 summary
Hyperparameters:
lstm_layers: 2
layers: 5
dropout: 0
sequences: 1
activation: leaky_relu
tuner/epochs: 9
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0001
Sc